In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from models_def import cnn2d,cnn_v1,cnn_v2,cudnn_gru,lstm_v1,cnn_gru,gru_v1
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import text, sequence

model_defines = [
    #[cnn2d,'cnn2d'],
    #[cnn_v1,'cnn_v1'],
    #[cnn_v2,'cnn_v2'],
    #[cudnn_gru,'cudnn_gru'],
    #[lstm_v1,'lstm_v1'],
    [cnn_gru,'cnn_gru'],
    [gru_v1,'gru_v1']
]

max_features = 40000
maxlen = 150

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)

Using TensorFlow backend.


(159571, 6)


In [2]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

# read word2vec
word_vec_dict = {}
with open('../glove.840B.300d.txt') as f:
    for line in f:
        v_list = line.strip().split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v

print(len(word_vec_dict))
print('Preparing embedding matrix')
EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
del word_vec_dict

(159571, 150) (153164, 150)
thorbjorgolfsson 84274
agw 13855
arsey 173664
camaro 46622
snarkily 43703
addem 153824
2196007
Preparing embedding matrix
Null word embeddings: 3388


In [3]:
from sklearn.metrics import log_loss,accuracy_score
from sklearn.model_selection import KFold
from keras import backend as K
import pickle
from keras.models import load_model
import gc

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def kf_train(model_func,fold_cnt=3,rnd=1,epo=10,batch=64):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = model_func(maxlen,nb_words,EMBEDDING_DIM,embedding_matrix,trainable_flag=True,comp=True)
        batch_size = batch
        epochs = epo
        file_path="best_model.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
        
        # clear
        del model
        gc.collect()
        K.clear_session()
        
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    eval_val(y,train_pred)
    return train_pred, test_pred

rnd_init = 100
for model_func,model_name in model_defines:
    for fold_k in [3,4]:
        print('==================================')
        if 'cnn' in model_name:
            epo = 8
            batch = 64
        else:
            epo = 10
            batch = 256
        train_pred,test_pred = kf_train(model_func,fold_k,rnd_init,epo,batch)
        output_feat = '../features/glove_{}_{}_feat.pkl'.format(model_name,fold_k)
        with open(output_feat,'wb') as fout:
            pickle.dump([train_pred,test_pred],fout)
        print(output_feat,'done')
        rnd_init += 1
        




Train on 106380 samples, validate on 53191 samples
Epoch 1/8
106380/106380 [==============================] - 165s 2ms/step - loss: 0.0559 - acc: 0.9799 - val_loss: 0.0430 - val_acc: 0.9833
Epoch 2/8
106380/106380 [==============================] - 164s 2ms/step - loss: 0.0388 - acc: 0.9848 - val_loss: 0.0419 - val_acc: 0.9837
Epoch 3/8
106380/106380 [==============================] - 164s 2ms/step - loss: 0.0326 - acc: 0.9870 - val_loss: 0.0450 - val_acc: 0.9828
Epoch 4/8
106380/106380 [==============================] - 164s 2ms/step - loss: 0.0273 - acc: 0.9890 - val_loss: 0.0484 - val_acc: 0.9826
Epoch 5/8
106380/106380 [==============================] - 164s 2ms/step - loss: 0.0233 - acc: 0.9908 - val_loss: 0.0515 - val_acc: 0.9821
Epoch 6/8
106380/106380 [==============================] - 164s 2ms/step - loss: 0.0194 - acc: 0.9924 - val_loss: 0.0564 - val_acc: 0.9813
Epoch 7/8
106380/106380 [==============================] - 164s 2ms/step - loss: 0.0162 - acc: 0.9938 - val_loss: 0

Epoch 5/8
119678/119678 [==============================] - 173s 1ms/step - loss: 0.0235 - acc: 0.9906 - val_loss: 0.0502 - val_acc: 0.9816
Epoch 6/8
119678/119678 [==============================] - 173s 1ms/step - loss: 0.0198 - acc: 0.9921 - val_loss: 0.0639 - val_acc: 0.9824
Epoch 7/8
119678/119678 [==============================] - 173s 1ms/step - loss: 0.0166 - acc: 0.9935 - val_loss: 0.0643 - val_acc: 0.9817
Epoch 8/8
119678/119678 [==============================] - 173s 1ms/step - loss: 0.0141 - acc: 0.9946 - val_loss: 0.0687 - val_acc: 0.9807
Train on 119678 samples, validate on 39893 samples
Epoch 1/8
119678/119678 [==============================] - 176s 1ms/step - loss: 0.0540 - acc: 0.9806 - val_loss: 0.0447 - val_acc: 0.9828
Epoch 2/8
119678/119678 [==============================] - 175s 1ms/step - loss: 0.0389 - acc: 0.9846 - val_loss: 0.0435 - val_acc: 0.9830
Epoch 3/8
119678/119678 [==============================] - 175s 1ms/step - loss: 0.0329 - acc: 0.9867 - val_loss: 0

106380/106380 [==============================] - 163s 2ms/step - loss: 0.0741 - acc: 0.9758 - val_loss: 0.0464 - val_acc: 0.9823
Epoch 2/10
106380/106380 [==============================] - 159s 1ms/step - loss: 0.0447 - acc: 0.9832 - val_loss: 0.0448 - val_acc: 0.9827
Epoch 3/10
106380/106380 [==============================] - 163s 2ms/step - loss: 0.0392 - acc: 0.9847 - val_loss: 0.0426 - val_acc: 0.9837
Epoch 4/10
106380/106380 [==============================] - 166s 2ms/step - loss: 0.0351 - acc: 0.9860 - val_loss: 0.0437 - val_acc: 0.9832
Epoch 5/10
106380/106380 [==============================] - 164s 2ms/step - loss: 0.0314 - acc: 0.9873 - val_loss: 0.0449 - val_acc: 0.9830
Epoch 6/10
106380/106380 [==============================] - 162s 2ms/step - loss: 0.0285 - acc: 0.9883 - val_loss: 0.0476 - val_acc: 0.9824
Epoch 7/10
106380/106380 [==============================] - 162s 2ms/step - loss: 0.0256 - acc: 0.9895 - val_loss: 0.0523 - val_acc: 0.9823
Epoch 8/10
106380/106380 [=====

-------------------------------
0 0.095838153169 0.964542429389
1 0.0223952950057 0.989935514599
2 0.0461760585698 0.982045609791
3 0.0128604891752 0.996998201428
4 0.0605214410149 0.97523986188
5 0.0209285071078 0.992260498461
final 0.0431199906737 0.983503685925
../features/glove_gru_v1_3_feat.pkl done
Train on 119678 samples, validate on 39893 samples
Epoch 1/10
119678/119678 [==============================] - 173s 1ms/step - loss: 0.0729 - acc: 0.9747 - val_loss: 0.0477 - val_acc: 0.9816
Epoch 2/10
119678/119678 [==============================] - 173s 1ms/step - loss: 0.0441 - acc: 0.9832 - val_loss: 0.0432 - val_acc: 0.9833
Epoch 3/10
119678/119678 [==============================] - 173s 1ms/step - loss: 0.0387 - acc: 0.9847 - val_loss: 0.0431 - val_acc: 0.9836
Epoch 4/10
119678/119678 [==============================] - 173s 1ms/step - loss: 0.0348 - acc: 0.9861 - val_loss: 0.0446 - val_acc: 0.9830
Epoch 5/10
119678/119678 [==============================] - 173s 1ms/step - loss: 0

Epoch 9/10
119678/119678 [==============================] - 171s 1ms/step - loss: 0.0204 - acc: 0.9918 - val_loss: 0.0609 - val_acc: 0.9816
Epoch 10/10
119678/119678 [==============================] - 171s 1ms/step - loss: 0.0187 - acc: 0.9926 - val_loss: 0.0631 - val_acc: 0.9818
Train on 119679 samples, validate on 39892 samples
Epoch 1/10
119679/119679 [==============================] - 171s 1ms/step - loss: 0.0693 - acc: 0.9773 - val_loss: 0.0459 - val_acc: 0.9827
Epoch 2/10
119679/119679 [==============================] - 171s 1ms/step - loss: 0.0432 - acc: 0.9833 - val_loss: 0.0435 - val_acc: 0.9832
Epoch 3/10
119679/119679 [==============================] - 170s 1ms/step - loss: 0.0381 - acc: 0.9850 - val_loss: 0.0442 - val_acc: 0.9825
Epoch 4/10
119679/119679 [==============================] - 170s 1ms/step - loss: 0.0346 - acc: 0.9861 - val_loss: 0.0456 - val_acc: 0.9824
Epoch 5/10
119679/119679 [==============================] - 170s 1ms/step - loss: 0.0312 - acc: 0.9873 - val